In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

# 처리할 파일 목록
files = [
    "지자체1.xml", "지자체2.xml", "지자체3.xml",
    "지자체4.xml", "지자체5.xml", "지자체6.xml"
]

rows = []

for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    
    for serv in root.iter("servList"):
        row = {
            "서비스ID": serv.findtext("servId", default=""),
            "서비스명": serv.findtext("servNm", default=""),
            "서비스 요약": serv.findtext("servDgst", default=""),
            "상세 링크": serv.findtext("servDtlLink", default=""),
            "시도명": serv.findtext("ctpvNm", default=""),
            "시군구명": serv.findtext("sggNm", default=""),
            "담당부서명": serv.findtext("bizChrDeptNm", default=""),
            "신청방법": serv.findtext("aplyMtdNm", default=""),
            "지원주기": serv.findtext("sprtCycNm", default=""),
            "지급방식": serv.findtext("srvPvsnNm", default=""),
            "생애주기": serv.findtext("lifeNmArray", default=""),
            "대상특성": serv.findtext("trgterIndvdlNmArray", default=""),
            "관심주제": serv.findtext("intrsThemaNmArray", default=""),
            "조회수": serv.findtext("inqNum", default=""),
            "최종수정일": serv.findtext("lastModYmd", default="")
        }
        rows.append(row)

# DataFrame으로 변환 후 CSV 저장
df = pd.DataFrame(rows)
df.to_csv("지자체_복지서비스_통합.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: 지자체_복지서비스_통합.csv")


✅ CSV 파일 저장 완료: 지자체_복지서비스_통합.csv


In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

# 중앙부처 XML 파일
file = "중앙부처.xml"

rows = []

tree = ET.parse(file)
root = tree.getroot()

for serv in root.iter("servList"):
    row = {
        "서비스ID": serv.findtext("servId", default=""),
        "서비스명": serv.findtext("servNm", default=""),
        "서비스 요약": serv.findtext("servDgst", default=""),
        "상세 링크": serv.findtext("servDtlLink", default=""),
        "소관부처": serv.findtext("jurMnofNm", default=""),
        "담당부서": serv.findtext("jurOrgNm", default=""),
        "대표연락처": serv.findtext("rprsCtadr", default=""),
        "신청가능여부": serv.findtext("onapPsbltYn", default=""),
        "신청방법": serv.findtext("aplyMtdNm", default=""),
        "지원주기": serv.findtext("sprtCycNm", default=""),
        "지급방식": serv.findtext("srvPvsnNm", default=""),
        "생애주기": serv.findtext("lifeArray", default=""),
        "대상특성": serv.findtext("trgterIndvdlArray", default=""),
        "관심주제": serv.findtext("intrsThemaArray", default=""),
        "조회수": serv.findtext("inqNum", default=""),
        "최초등록일": serv.findtext("svcfrstRegTs", default="")
    }
    rows.append(row)

# DataFrame 변환 후 CSV 저장
df = pd.DataFrame(rows)
df.to_csv("중앙부처_복지서비스.csv", index=False, encoding="utf-8-sig")

print("✅ 중앙부처 CSV 저장 완료: 중앙부처_복지서비스.csv")


✅ 중앙부처 CSV 저장 완료: 중앙부처_복지서비스.csv


In [3]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time

# 1. 기존 서비스 목록 CSV 로드
df = pd.read_csv("지자체_복지서비스_통합.csv")
service_ids = df["서비스ID"].dropna().tolist()

# 2. API 기본정보
API_KEY = "6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7"
BASE_URL = "https://apis.data.go.kr/B554287/LocalGovernmentWelfareInformations/LcgvWelfareInfo"

# 3. 결과 리스트
rows = []

# 4. 상세조회 반복
for i, sid in enumerate(service_ids, start=1):
    params = {"serviceKey": API_KEY, "servId": sid}
    try:
        r = requests.get(BASE_URL, params=params, timeout=10)
        if r.status_code == 200:
            root = ET.fromstring(r.text)

            # servList 대신 상세조회는 servDtl (혹은 servInfo 같은 태그) 안에 상세 데이터 있음
            # 실제 응답 XML 구조에 맞게 태그명 확인 필요
            service = root.find(".//servDtl") or root.find(".//wantedDetail") or root

            row = {
                "서비스ID": service.findtext("servId", ""),
                "서비스명": service.findtext("servNm", ""),
                "서비스 요약": service.findtext("servDgst", ""),
                "상세 내용": service.findtext("servDtlCont", ""),
                "소관부처": service.findtext("jurMnofNm", ""),
                "담당부서": service.findtext("jurOrgNm", ""),
                "대표연락처": service.findtext("rprsCtadr", ""),
                "신청방법": service.findtext("aplyMtdNm", ""),
                "지원주기": service.findtext("sprtCycNm", ""),
                "지급방식": service.findtext("srvPvsnNm", ""),
                "생애주기": service.findtext("lifeNmArray", ""),
                "대상특성": service.findtext("trgterIndvdlNmArray", ""),
                "관심주제": service.findtext("intrsThemaNmArray", ""),
                "최종수정일": service.findtext("lastModYmd", "")
            }
            rows.append(row)
            print(f"[{i}/{len(service_ids)}] {sid} 처리 완료")
        else:
            print(f"[{i}] {sid} 요청 실패 (status {r.status_code})")
    except Exception as e:
        print(f"[{i}] {sid} 에러: {e}")
    
    time.sleep(0.3)  # API 호출 간격 (서버 부하/제한 회피용)

# 5. DataFrame 저장
df_detail = pd.DataFrame(rows)
df_detail.to_csv("지자체_복지서비스_상세병합.csv", index=False, encoding="utf-8-sig")

print("✅ 상세조회 CSV 저장 완료: 지자체_복지서비스_상세병합.csv")


[1] WLF00006172 에러: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B554287/LocalGovernmentWelfareInformations/LcgvWelfareInfo?serviceKey=6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7&servId=WLF00006172 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1032)')))
[2] WLF00005696 에러: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B554287/LocalGovernmentWelfareInformations/LcgvWelfareInfo?serviceKey=6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7&servId=WLF00005696 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1032)')))
[3] WLF00006171 에러: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B554287/LocalGovernmentWelfareInformations/LcgvWelfareInfo?serviceKey=6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7&

KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# 1. Selenium 옵션 설정
options = Options()
options.add_argument("--headless=new")   # 창 안 띄우고 실행
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

# 2. API 엔드포인트
API_KEY = "여기에_본인_API키"
servId = "WLF00006172"
url = f"https://apis.data.go.kr/B554287/LocalGovernmentWelfareInformations/LcgvWelfareInfo?serviceKey={API_KEY}&servId={servId}"

# 3. 페이지 열기
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# 4. XML 텍스트 가져오기
xml_text = driver.find_element("tag name", "pre").text   # 대부분 <pre> 안에 XML 출력됨
print(xml_text[:500])  # 앞부분만 출력

# 5. 저장하기
with open(f"{servId}.xml", "w", encoding="utf-8") as f:
    f.write(xml_text)

driver.quit()


[1] 요청: WLF00006172
[1] WLF00006172 에러 발생: Message: no such element: Unable to locate element: {"method":"tag name","selector":"pre"}
  (Session info: chrome=140.0.7339.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff72d1630f5+79493]
	GetHandleVerifier [0x0x7ff72d163150+79584]
	(No symbol) [0x0x7ff72cee01ba]
	(No symbol) [0x0x7ff72cf38067]
	(No symbol) [0x0x7ff72cf3832c]
	(No symbol) [0x0x7ff72cf8be27]
	(No symbol) [0x0x7ff72cf6074f]
	(No symbol) [0x0x7ff72cf88b8b]
	(No symbol) [0x0x7ff72cf604e3]
	(No symbol) [0x0x7ff72cf28e92]
	(No symbol) [0x0x7ff72cf29c63]
	GetHandleVerifier [0x0x7ff72d420dbd+2954061]
	GetHandleVerifier [0x0x7ff72d41b02a+2930106]
	GetHandleVerifier [0x0x7ff72d43b357+3061991]
	GetHandleVerifier [0x0x7ff72d17d60e+187294]
	GetHandleVerifier [0x0x7ff72d18557f+219919]
	GetHandleVerifier [0x0x7ff72d16c294+116772]
	GetHandleVerifier [0x

In [11]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities.CHROME
caps["goog:loggingPrefs"] = {"performance": "ALL"}

options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(), options=options, desired_capabilities=caps)


TypeError: WebDriver.__init__() got an unexpected keyword argument 'desired_capabilities'

In [26]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time
import warnings

# SSL 경고 메시지를 무시하도록 설정
from requests.packages.urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)

# --- 🚨 사용자 수정 필요 (1/2): CSV 파일 정보 ---
CSV_FILE_NAME = '지자체_복지서비스_통합.csv'
SERVICE_ID_COLUMN = '서비스ID' # CSV 파일의 서비스 ID 컬럼명

# --- 🚨 사용자 수정 필요 (2/2): API 키 ---
API_KEY = '6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7' # 👉 발급받은 인증키를 여기에 넣으세요.

# --- 올바른 API 주소 ---
BASE_URL = "https://apis.data.go.kr/B554287/LcgvWelfaredetailed/LcgvWelfaredetailed"

def read_service_ids_from_csv(filename, column_name):
    """CSV 파일에서 서비스 ID 목록을 읽어옵니다."""
    try:
        try:
            df = pd.read_csv(filename, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(filename, encoding='cp949')
        
        if column_name not in df.columns:
            print(f"❌ 오류: CSV 파일에 '{column_name}' 컬럼이 없습니다.")
            print(f"👉 실제 컬럼명: {df.columns.tolist()}")
            return None
            
        service_ids = df[column_name].dropna().astype(str).tolist()
        print(f"✅ CSV에서 {len(service_ids)}개의 서비스 ID를 읽었습니다.")
        return service_ids
    except FileNotFoundError:
        print(f"❌ 오류: '{filename}' 파일을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"❌ CSV 파일을 읽는 중 오류: {e}")
        return None

def fetch_and_parse_data(service_ids):
    """API를 호출하여 XML 데이터를 파싱하고 리스트로 반환합니다."""
    rows = []
    print("\n✅ [2단계] API 상세 정보 요청을 시작합니다.")
    
    for i, sid in enumerate(service_ids, start=1):
        url = f"{BASE_URL}?serviceKey={API_KEY}&servID={sid}"
        print(f"[{i}/{len(service_ids)}] 요청: {sid}")

        try:
            # ✨ SSL 인증서 검증을 건너뛰는 verify=False 옵션 추가
            resp = requests.get(url, timeout=15, verify=False)
            
            if resp.status_code != 200:
                print(f"  ❌ 오류: 상태코드 {resp.status_code}")
                continue

            root = ET.fromstring(resp.text)
            
            # resultCode가 '00' (정상)이 아닌 경우 오류 메시지 출력
            result_code = root.findtext(".//resultCode", "")
            if result_code not in ['0', '00']:
                result_msg = root.findtext(".//resultMessage", "알 수 없는 오류")
                print(f"  ❌ API 오류 ({result_code}): {result_msg}")
                continue

            # XML 데이터 파싱
            row = {
                "서비스ID": root.findtext("servId", ""),
                "서비스명": root.findtext("servNm", ""),
                "서비스 요약": root.findtext("servDgst", ""),
                "지원대상": root.findtext("sprtTrgtCn", "").strip(),
                "선정기준": root.findtext("slctCritCn", "").strip(),
                "지원내용": root.findtext("alwServCn", "").strip(),
                "신청방법": root.findtext("aplyMtdCn", "").strip(),
                "지원주기": root.findtext("sprtCycNm", ""),
                "지급방식": root.findtext("srvPvsnNm", ""),
                "담당부서": root.findtext("bizChrDeptNm", ""),
                "대상특성": root.findtext("trgterIndvdlNmArray", ""),
                "관심주제": root.findtext("intrsThemaNmArray", ""),
                "최종수정일": root.findtext("lastModYmd", "")
            }
            rows.append(row)
            
        except requests.exceptions.RequestException as e:
            print(f"  ❌ 네트워크 에러: {e}")
        except ET.ParseError as e:
            print(f"  ❌ XML 파싱 에러: {e}")
        except Exception as e:
            print(f"  ❌ 알 수 없는 에러: {e}")
        
        time.sleep(0.1) # API 서버 과부하 방지

    print("🎉 상세 정보 요청 완료")
    return rows

def save_to_csv(data, filename="지자체_복지서비스_상세.csv"):
    """결과 데이터를 CSV 파일로 저장합니다."""
    if not data:
        print("⚠️ 저장할 데이터가 없습니다.")
        return
        
    print(f"\n✅ [3단계] 결과를 '{filename}' 파일로 저장합니다.")
    df_out = pd.DataFrame(data)
    df_out.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"💾 저장 완료!")

# --- 메인 실행 ---
if __name__ == "__main__":
    # 1. CSV에서 전체 서비스 ID 읽기
    all_ids = read_service_ids_from_csv(CSV_FILE_NAME, SERVICE_ID_COLUMN)
    
    if all_ids:
        # 🧪 테스트를 위해 10개만 사용
        ids_to_process = all_ids[:10] 
        
        # 2. API 호출 및 데이터 파싱
        results = fetch_and_parse_data(ids_to_process)
        
        # 3. CSV로 저장
        save_to_csv(results)
        
    print("\n✨ 모든 작업 종료")

✅ CSV에서 4555개의 서비스 ID를 읽었습니다.

✅ [2단계] API 상세 정보 요청을 시작합니다.
[1/10] 요청: WLF00006172
  ❌ 네트워크 에러: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B554287/LcgvWelfaredetailed/LcgvWelfaredetailed?serviceKey=6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7&servID=WLF00006172 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1032)')))
[2/10] 요청: WLF00005696
  ❌ 네트워크 에러: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B554287/LcgvWelfaredetailed/LcgvWelfaredetailed?serviceKey=6d4c8b3d1ac688d415302faaf9fd536a3db85e19308ebc90cea7302c56fc9de7&servID=WLF00005696 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1032)')))
[3/10] 요청: WLF00006171
  ❌ 네트워크 에러: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B554287/LcgvWelfaredetailed/LcgvWelfaredetailed?s